In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import json
import config
from checklist.test_suite import TestSuite
import pickle
from data_sets.suite_utils import *
# from utils.results import *
from pathlib import Path

In [ ]:
import string
import re

def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""

    def remove_articles(text):
        return re.sub(r"\b(a|an|the)\b", " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()
    return white_space_fix(remove_articles(remove_punc(lower(s))))

In [ ]:
task = "pi"

In [ ]:
results_path = Path(f"./results/{task}/suite/")

In [ ]:
files = {}
for children in results_path.rglob(f"*.txt"):
    files[children.name[:-4]] = children

In [ ]:
if task == "sa":
    suite_path = config.sa_suite
elif task == "pi":
    suite_path = config.pi_suite
elif task == "rc":
    suite_path = config.rc_suite
suite = TestSuite.from_file(suite_path)

In [ ]:
if task == "rc":
    for test in suite.tests.values():
        if test.labels is not None:
            for idx, labels in enumerate(test.labels):
                if type(labels) == str:
                    test.labels[idx] = normalize_answer(labels)
                elif type(labels) == list:
                    for sample_labels in labels:
                        test.labels[idx] = [normalize_answer(label) for label in labels]
                else:
                    print(type(labels))

In [ ]:
with open(suite_path.parent/"test_ids.pkl", "rb") as file:
    ids = pickle.load(file)

In [ ]:
fix_run_idxs(suite, ids)

In [ ]:
for model, file in files.items():
    output_path = Path(f"{results_path}/{model}.csv")
    hits_path = Path(f"{results_path}/{model}_hits.json")
    if output_path.exists() and hits_path.exists():
        print(f"Jumping saved {model} scores")
        continue
    if task == "sa":
        try:
            suite.run_from_file(file, overwrite=True)
        except IndexError:
            print(f"Problem with file {file}")
    elif task == "pi":
        suite.run_from_file(file, file_format="pred_only", overwrite=True)
    elif task == "rc":
        suite.run_from_file(file, file_format="pred_only", overwrite=True)
    results, all_hits = get_pass_rates(suite)
    df = pd.DataFrame.from_dict({model: results}, orient="index")
    df["avg"] = df.mean(axis=1)
    if not output_path.exists():
        df.to_csv(output_path)
    with open(hits_path, "w") as file:
        json.dump(all_hits, file)